Set Up

In [1]:
#Import Dependencies
import numpy as np
import pandas as pd
import requests
import gmaps
import gmaps.datasets
import os
import json
import time
import matplotlib.pyplot as plt


#Import API key
from resource import g_api_key



In [3]:
#import city dataframe csv created in Weather.py
city_df = pd.read_csv("output/city_data.csv")

In [4]:
city_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed
0,hithadhoo,-0.6000,73.0833,82.44,71,28,10.71
1,monte azul,-15.1550,-42.8747,73.45,73,77,2.98
2,ushuaia,-54.8000,-68.3000,40.66,87,100,13.80
3,bambous virieux,-20.3428,57.7575,80.22,83,100,18.57
4,qaanaaq,77.4840,-69.3632,-0.42,62,0,7.34


In [13]:
city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        552 non-null    object 
 1   Lat         552 non-null    float64
 2   Long        552 non-null    float64
 3   Max Temp    552 non-null    float64
 4   Humidity    552 non-null    int64  
 5   Cloudiness  552 non-null    int64  
 6   Wind Speed  552 non-null    float64
dtypes: float64(4), int64(2), object(1)
memory usage: 30.3+ KB


Humidity Heatmap


In [5]:
#Configure gmaps
gmaps.configure(api_key=g_api_key)

#store city
city = city_df[["Lat", "Long"]]

#store humidity
humidity = city_df["Humidity"]


In [6]:
#plot heatmap
fig = gmaps.figure(center=(46.0,-5.0), zoom_level=8)

#create heatlayer
heat_layer = gmaps.heatmap_layer(city, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

#Add layer 

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

Narrow list of cities to those with a max temperature between 65 & 87, cloudiness less than 70%, humidity less than 70% and wind speed less than 10mph. 

In [36]:
pleasant_weather = city_df.loc[(city_df["Max Temp"]>65)\
                            &(city_df["Max Temp"]<87)\
                            &(city_df["Cloudiness"]<70)
                            &(city_df["Humidity"]<70)\
                            &(city_df["Wind Speed"]<=10)].dropna()
                        

In [38]:
pleasant_weather

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed
78,geraldton,-28.7667,114.6000,66.61,63,50,8.05
81,raga,8.4596,25.6780,83.03,14,7,2.19
116,kutum,14.2000,24.6667,76.23,11,19,8.16
136,yambio,4.5721,28.3955,79.97,36,54,2.95
142,salalah,17.0151,54.0924,75.29,69,0,3.44
143,tessalit,20.1986,1.0114,75.02,7,56,9.69
145,nalgonda,17.0500,79.2667,71.28,66,1,4.36
151,junin,-34.5838,-60.9433,79.21,51,57,7.56
241,magugu,-4.0167,35.7667,69.85,67,60,2.95
243,bafoulabe,13.8065,-10.8321,86.85,11,66,5.73


Find Hotels near the "Pleasant Weather" Cities

In [45]:
#create a dataframe for the hotels including: City, Row for Hotel Name
hotel_df = pleasant_weather.loc[:,("Lat","Long", "City")]
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Lat,Long,City,Hotel Name
78,-28.7667,114.6000,geraldton,
81,8.4596,25.6780,raga,
116,14.2000,24.6667,kutum,
136,4.5721,28.3955,yambio,
142,17.0151,54.0924,salalah,


In [53]:
#set up base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#set up a dictionary of hotels within 5000 meters (3.1 miles) of each city 
parameters = {"radius": 5000, 
              "types" : "hotel",
              "keyword": "hotel",
              "key": g_api_key          
             }

#use the lat & long in the pleasant weather df to identify hotels
for index, row in hotel_df.iterrows():
    hotel_lat = row["Lat"]
    hotel_long = row["Long"]
    hotel_city = row["City"]
    
    
    #change location for each iteration but leave parameters in place
    parameters["location"] = f"{hotel_lat},{hotel_long}"
    
    print(f"Retrieving results for index {index}: {hotel_city}")
    response = requests.get(base_url, parameters).json()
    results = response["results"]
    
    #Save the hotel name to the data frame, add error handling
    try:
        print(f"Nearest hotel to {hotel_city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
              
    except:
        print(f"No hotel found")
        print("----------------------------------")
        #add delay to avoid errors
        time.sleep(1)
        print("--------------------------------Ending Search")
   

Retrieving results for index 78: geraldton
Nearest hotel to geraldton is The Geraldton Hotel.
Retrieving results for index 81: raga
No hotel found
----------------------------------
--------------------------------Ending Search
Retrieving results for index 116: kutum
No hotel found
----------------------------------
--------------------------------Ending Search
Retrieving results for index 136: yambio
Nearest hotel to yambio is Tourist Hotel.
Retrieving results for index 142: salalah
Nearest hotel to salalah is Salalah Gardens Hotel.
Retrieving results for index 143: tessalit
No hotel found
----------------------------------
--------------------------------Ending Search
Retrieving results for index 145: nalgonda
Nearest hotel to nalgonda is Hotel Manorama..
Retrieving results for index 151: junin
Nearest hotel to junin is Hotel Avenida.
Retrieving results for index 241: magugu
No hotel found
----------------------------------
--------------------------------Ending Search
Retrieving res

In [54]:
hotel_df


,Lat,Long,City,Hotel Name
78,-28.7667,114.6000,geraldton,The Geraldton Hotel
81,8.4596,25.6780,raga,
116,14.2000,24.6667,kutum,
136,4.5721,28.3955,yambio,Tourist Hotel
142,17.0151,54.0924,salalah,Salalah Gardens Hotel
143,20.1986,1.0114,tessalit,
145,17.0500,79.2667,nalgonda,Hotel Manorama.
151,-34.5838,-60.9433,junin,Hotel Avenida
241,-4.0167,35.7667,magugu,
243,13.8065,-10.8321,bafoulabe,Madi Sissoko


In [56]:
#add hotels to heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]


In [59]:
#add markers on top of the heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

fig



Figure(layout=FigureLayout(height='420px'))